<a href="https://colab.research.google.com/github/pascalghanimi/Injury-Prediction-in-Runners/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

all_data = np.load("all_data.npz")

X_train_injured = all_data["X_train_injured"]
X_val_injured = all_data["X_val_injured"]
X_test_injured = all_data["X_test_injured"]
Y_train_labels_injured = all_data["Y_train_labels_injured"]
Y_val_labels_injured = all_data["Y_val_labels_injured"]
Y_test_labels_injured = all_data["Y_test_labels_injured"]
X_train_uninjured = all_data["X_train_uninjured"]
X_val_uninjured = all_data["X_val_uninjured"]
X_test_uninjured = all_data["X_test_uninjured"]
Y_train_labels_uninjured = all_data["Y_train_labels_uninjured"]
Y_val_labels_uninjured = all_data["Y_val_labels_uninjured"]
Y_test_labels_uninjured = all_data["Y_test_labels_uninjured"]


weeks_data = np.load("weeks_data.npz")

X_train_injured_weeks = weeks_data["X_train_injured_weeks"]
X_val_injured_weeks = weeks_data["X_val_injured_weeks"]
X_test_injured_weeks = weeks_data["X_test_injured_weeks"]
Y_train_labels_injured_weeks = weeks_data["Y_train_labels_injured_weeks"]
Y_val_labels_injured_weeks = weeks_data["Y_val_labels_injured_weeks"]
Y_test_labels_injured_weeks = weeks_data["Y_test_labels_injured_weeks"]
X_train_uninjured_weeks = weeks_data["X_train_uninjured_weeks"]
X_val_uninjured_weeks = weeks_data["X_val_uninjured_weeks"]
X_test_uninjured_weeks = weeks_data["X_test_uninjured_weeks"]
Y_train_labels_uninjured_weeks = weeks_data["Y_train_labels_uninjured_weeks"]
Y_val_labels_uninjured_weeks = weeks_data["Y_val_labels_uninjured_weeks"]
Y_test_labels_uninjured_weeks = weeks_data["Y_test_labels_uninjured_weeks"]


days_data = np.load("days_data.npz")

X_train_injured_days = days_data["X_train_injured_days"]
X_val_injured_days = days_data["X_val_injured_days"]
X_test_injured_days = days_data["X_test_injured_days"]
Y_train_labels_injured_days = days_data["Y_train_labels_injured_days"]
Y_val_labels_injured_days = days_data["Y_val_labels_injured_days"]
Y_test_labels_injured_days = days_data["Y_test_labels_injured_days"]
X_train_uninjured_days = days_data["X_train_uninjured_days"]
X_val_uninjured_days = days_data["X_val_uninjured_days"]
X_test_uninjured_days = days_data["X_test_uninjured_days"]
Y_train_labels_uninjured_days = days_data["Y_train_labels_uninjured_days"]
Y_val_labels_uninjured_days = days_data["Y_val_labels_uninjured_days"]
Y_test_labels_uninjured_days = days_data["Y_test_labels_uninjured_days"]


print(X_train_injured[0])
print("Shape X_train_injured", X_train_injured.shape)
print("Size X_train_injured", X_train_injured.size)

print("Shape X_train_injured_weeks", X_train_injured_weeks.shape)
print("Shape X_train_injured_days", X_train_injured_days.shape)


[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.00000000e+00  8.00000000e-02
  0.00000000e+00  1.80000000e-01  1.00000000e+00  1.15000000e+01
  5.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.00000000e-01  0.00000000e+00  1.80000000e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00000000e-02
 -1.00000000e-02 -1.00000000e-02  1.00000000e+00  4.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.00000000e-01  0.00000000e+00  1.70000000e-01
  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.25000000e+00  9.00000000e-02
  0.00000000e+00  1.70000000e-01  1.00000000e+00  4.90000000e+00
  0.00000000e+00  0.00000000e+00  9.00000000e-01  0.00000000e+00
  0.00000000e+00  1.00000000e-01  0.00000000e+00  1.70000000e-01
  1.00000000e+00  1.45000

In [22]:
# Validierungs- und Testdatensatz vorbereiten und mischen
X_val = np.concatenate([X_val_injured, X_val_uninjured])
Y_val = np.concatenate([Y_val_labels_injured, Y_val_labels_uninjured])
val_indices = np.arange(len(Y_val))
np.random.shuffle(val_indices)
X_val = X_val[val_indices]
Y_val = Y_val[val_indices]

X_test = np.concatenate([X_test_injured, X_test_uninjured])
Y_test = np.concatenate([Y_test_labels_injured, Y_test_labels_uninjured])
test_indices = np.arange(len(Y_test))
np.random.shuffle(test_indices)
X_test = X_test[test_indices]
Y_test = Y_test[test_indices]

In [30]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

# Daten vorbereiten und mischen
X_val = np.concatenate([X_val_injured, X_val_uninjured])
Y_val = np.concatenate([Y_val_labels_injured, Y_val_labels_uninjured])
val_indices = np.arange(len(Y_val))
np.random.shuffle(val_indices)
X_val = X_val[val_indices]
Y_val = Y_val[val_indices]

X_test = np.concatenate([X_test_injured, X_test_uninjured])
Y_test = np.concatenate([Y_test_labels_injured, Y_test_labels_uninjured])
test_indices = np.arange(len(Y_test))
np.random.shuffle(test_indices)
X_test = X_test[test_indices]
Y_test = Y_test[test_indices]

# Scaler GLOBAL auf allen Trainingsdaten initialisieren
scaler = StandardScaler()
scaler.fit(np.concatenate([X_train_injured, X_train_uninjured]))

# Funktion für dynamische Batcherstellung
def get_training_batch(
    X_train_injured,
    Y_train_labels_injured,
    X_train_uninjured,
    Y_train_labels_uninjured,
    batch_size
):
    injured_indices = np.random.choice(len(X_train_injured), batch_size // 2, replace=True)
    uninjured_indices = np.random.choice(len(X_train_uninjured), batch_size // 2, replace=True)

    X_batch = np.concatenate([
        X_train_injured[injured_indices],
        X_train_uninjured[uninjured_indices]
    ])
    Y_batch = np.concatenate([
        Y_train_labels_injured[injured_indices],
        Y_train_labels_uninjured[uninjured_indices]
    ])

    # Scaler NUR transformieren (nicht fit_transform!)
    X_batch = scaler.transform(X_batch)

    return X_batch, Y_batch

# Modelparameter
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "eta": 0.05,
    "max_depth": 3,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "n_estimators": 512,
    "lambda": 1.0,
    "alpha": 0.5
}

# Trainingskonfiguration
num_models = 9
models = []
num_epochs = 20
batch_size = 256

# Validierungs- und Testdaten skalieren (einmalig)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Training
for model_idx in range(num_models):
    print(f"\nTraining Modell {model_idx + 1}/{num_models}")
    model = xgb.XGBClassifier(**params)
    highest_auc = 0

    for epoch in range(num_epochs):
        # NEUER Batch in jeder Epoche
        X_batch, Y_batch = get_training_batch(
            X_train_injured,
            Y_train_labels_injured,
            X_train_uninjured,
            Y_train_labels_uninjured,
            batch_size
        )

        model.fit(X_batch, Y_batch, eval_set=[(X_val_scaled, Y_val)], verbose=False)
        Y_pred_val = model.predict_proba(X_val_scaled)[:, 1]
        auc = roc_auc_score(Y_val, Y_pred_val)

        print(f"Epoch {epoch + 1}/{num_epochs} | Val-AUC: {auc:.4f}")

        if auc > highest_auc:
            highest_auc = auc

    # Finale Testbewertung
    Y_pred_test = model.predict_proba(X_test_scaled)[:, 1]
    test_auc = roc_auc_score(Y_test, Y_pred_test)
    print(f"\nModell {model_idx + 1} Ergebnisse:")
    print(f"Beste Val-AUC: {highest_auc:.4f}")
    print(f"Test-AUC: {test_auc:.4f}")
    models.append(model)


Training Modell 1/9
Epoch 1/20 | Val-AUC: 0.7386
Epoch 2/20 | Val-AUC: 0.6105
Epoch 3/20 | Val-AUC: 0.5395
Epoch 4/20 | Val-AUC: 0.7215
Epoch 5/20 | Val-AUC: 0.4650
Epoch 6/20 | Val-AUC: 0.6060
Epoch 7/20 | Val-AUC: 0.5559
Epoch 8/20 | Val-AUC: 0.4515
Epoch 9/20 | Val-AUC: 0.7136
Epoch 10/20 | Val-AUC: 0.4399
Epoch 11/20 | Val-AUC: 0.5503
Epoch 12/20 | Val-AUC: 0.6202
Epoch 13/20 | Val-AUC: 0.6423
Epoch 14/20 | Val-AUC: 0.6472
Epoch 15/20 | Val-AUC: 0.6297
Epoch 16/20 | Val-AUC: 0.6912
Epoch 17/20 | Val-AUC: 0.4565
Epoch 18/20 | Val-AUC: 0.6293
Epoch 19/20 | Val-AUC: 0.6165
Epoch 20/20 | Val-AUC: 0.5661

Modell 1 Ergebnisse:
Beste Val-AUC: 0.7386
Test-AUC: 0.5534

Training Modell 2/9
Epoch 1/20 | Val-AUC: 0.8125
Epoch 2/20 | Val-AUC: 0.3995
Epoch 3/20 | Val-AUC: 0.5914
Epoch 4/20 | Val-AUC: 0.4805
Epoch 5/20 | Val-AUC: 0.3957
Epoch 6/20 | Val-AUC: 0.7107
Epoch 7/20 | Val-AUC: 0.5334
Epoch 8/20 | Val-AUC: 0.6948
Epoch 9/20 | Val-AUC: 0.5707
Epoch 10/20 | Val-AUC: 0.5722
Epoch 11/20 | V

KeyboardInterrupt: 